In [22]:
import pandas as pd
import numpy as np                  

In [23]:
df = pd.read_csv('911_calls_for_service.csv') # Parse the callDatetime column

In [24]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [25]:
df.head(14)

,callDateTime,priority,district,description,callNumber,incidentLocation,location
0,2015-07-13 10:41:00,Medium,CD,SEE TEXT,P151941002,0 N CALVERT ST,"(39.2899299,-76.6123462)"
1,2015-07-13 10:47:00,Medium,CD,911/NO VOICE,P151941003,600 E FAYETTE ST,"(39.2906737,-76.6071600)"
2,2015-07-13 10:42:00,Medium,CD,911/NO VOICE,P151941004,200 E BALTIMORE ST,"(39.2898910,-76.6120720)"
3,2015-07-13 10:45:00,Low,CD,PRKG COMPLAINT,P151941005,800 PARK AV,"(39.2985163,-76.6184754)"
4,2015-07-13 10:46:00,Medium,SW,AUTO THEFT,P151941006,3500 CLIFTON AV,"(39.3112130,-76.6763150)"
5,2015-07-13 10:47:00,Medium,ND,FAMILY DISTURB,P151941007,2700 N CALVERT ST,"(39.3208510,-76.6147390)"
6,2015-07-13 10:46:00,High,WD,SILENT ALARM,P151941008,2100 W NORTH AV,"(39.3097096,-76.6513109)"
7,2015-07-13 10:49:00,Low,SW,AUTO ACCIDENT,P151941010,3100 WILKENS AV,"(39.2756929,-76.6664179)"
8,2015-07-13 10:48:00,Medium,NE,FAMILY DISTURB,P151941011,4800 GILRAY DR,"(39.3483090,-76.5768440)"
9,2015-07-13 10:49:00,Medium,ND,NARCOTICSOutside,P151941012,W GARRISON AV/PIMLICO RD,"(39.349653,-76.669145)"


In [26]:
# no missing data for 'description'
df.isnull().sum()

callDateTime           0
priority            6655
district               0
description            0
callNumber             0
incidentLocation       0
location               0
dtype: int64

In [27]:
df['description']=df['description'].str.upper()

In [28]:
df.loc[[140242]]

,callDateTime,priority,district,description,callNumber,incidentLocation,location
140242,2015-08-28 12:45:00,Low,NE,.,P152401388,WALKER AV/LOCH RAVEN BL,"(39.370120,-76.583377)"


In [29]:
#Replacing all non-alphanumeric characters with empty string
#df = df.replace({'description': '[\W]'},'',regex=True)
df = df.replace({'description': '[^a-zA-Z0-9/ _]'},'',regex=True)

In [30]:
df.description = df.description.str.replace('/',' ')
df.description = df.description.str.replace('  ',' ')

In [31]:
df.loc[[140242]]

,callDateTime,priority,district,description,callNumber,incidentLocation,location
140242,2015-08-28 12:45:00,Low,NE,,P152401388,WALKER AV/LOCH RAVEN BL,"(39.370120,-76.583377)"


In [32]:
df.description = df.description.str.strip()

In [33]:
df.description[df.description==''].count()

115

In [34]:
df.loc[df.description=='','description'] = 'NO DESC'

In [35]:
grouped = df.groupby('description').count().reset_index()
grouped.sort_values('priority', ascending=False,inplace=True)
grouped[grouped.priority>1000].count()

description         115
callDateTime        115
priority            115
district            115
callNumber          115
incidentLocation    115
location            115
dtype: int64

In [36]:
grouped[grouped.priority>1000].head(n=10)

,description,callDateTime,priority,district,callNumber,incidentLocation,location
221,911 NO VOICE,489758,489752,489758,489758,489758,489758
1821,DISORDERLY,250189,249481,250189,250189,250189,250189
5746,TRAFFIC STOP,167912,167912,167912,167912,167912,167912
1508,COMMON ASSAULT,125568,123685,125568,125568,125568,125568
676,AUTO ACCIDENT,116050,115980,116050,116050,116050,116050
3851,NARCOTICSOUTSIDE,99549,99455,99549,99549,99549,99549
4117,OTHER,91515,91504,91515,91515,91515,91515
5349,SILENT ALARM,84249,84163,84249,84249,84249,84249
2248,FAMILY DISTURB,72673,72470,72673,72673,72673,72673
5040,REPAIRS SERVICE,67050,67050,67050,67050,67050,67050


In [37]:
grouped[grouped.priority>1000].to_csv('911_calls_descriptions.csv')

In [38]:
df.description.replace({ #FINDING AND REPLACING THE DESCRIPTION
    'INVESTIFATE': 'INVESTIGATE',
    'BAIL-OUT': 'BAILOUT',
    'BALOUT': 'BAILOUT',
    ' BAIL OUT':'BAILOUT',
    'BAILED OUT': 'BAILOUT',
    ' ASSAULT': 'ASSAULT',
    ' ASSAULTED': 'ASSAULT',
    'Y': 'YELLING FOR HELP',
    'YELLING HELP': 'YELLING FOR HELP',
    'YP':'YELLING FOR HELP',
    'ECORT':'ESCORT',
    'WROMG NUMBER': 'WRONG NUMBER',
    'WRONG UNMBER': 'WRONG NUMBER',
    'Wanted on Warr': 'Warrant Service',
    'YELLING FOR HELP':'YELL FOR HELP',
    'YELL 4 HELP': 'YELL FOR HELP',
    'WRECKLESS DRVR': 'WRECKLESSDRIVER',''
    ' BAIL-0UT': 'BAILOUT',
    ' BAILOUT': 'BAILOUT',
    ' BAIL-OUT': 'BAILOUT',
    'WRECKLESS DRVING': 'WRECKLESSDRIVER',
    'WRECKLESS DRV': 'WRECKLESSDRIVER',
    'WRECKLESS DRIVR': 'WRECKLESSDRIVER',
    'WRECKLESS DRIVIN': 'WRECKLESSDRIVER',
    'WRECKLESS DRIVER':  'WRECKLESSDRIVER',
    'WRECKLESS DRIVE': 'WRECKLESSDRIVER',
    'WRECFKLESS DRIVE':'WRECKLESSDRIVER',
    'WRCKLSS DRIVER': 'WRECKLESSDRIVER',
    'WRCKLESS DRIVER': 'WRECKLESSDRIVER',
    ' WRECKLESS DRIVE':'WRECKLESSDRIVER',
    'WRECKLESS': 'WRECKLESSDRIVER',
    'WRECKLES DRIVING': 'WRECKLESSDRIVER',
    'WRECKLES DRIVERS': 'WRECKLESSDRIVER',
    'WRECKLES DRIVER': 'WRECKLESSDRIVER',
    '`INVEST': 'INVESTIGATE',
    '`INVESTIGATE': 'INVESTIGATE',
    'iNVESTIGATE': 'INVESTIGATE',
    '*#TRAFIC ARREST': '*#TRAFFIC ARREST',
    '&75': '& 75',
    '911/No Voice': '911/NO  VOICE',
    '911HANGUP': '911/HANGUP',
    '911 HANGUP': '911/HANGUP',
    'FAMILY DISTURBAN': 'FAMILY DISTURB',
    'FAMILY DISTURBP': 'FAMILY DISTURB',
    'AUTO ACC/INJURY': 'AUTO ACCIDENT',
    '*INVESTIGATE' : 'INVESTIGATE',
    '*TRAFFIC CONTROL'    : 'TRAFFIC CONTROL',
    'CHECK WELLBEING': 'CHECK WELL BEING'}, inplace = True)

In [39]:
df.to_csv('911_calls_for_service_description.csv')

In [40]:
df = pd.read_csv('911_calls_for_service_description.csv')

In [41]:
df.head()

,Unnamed: 0,callDateTime,priority,district,description,callNumber,incidentLocation,location
0,0,2015-07-13 10:41:00,Medium,CD,SEE TEXT,P151941002,0 N CALVERT ST,"(39.2899299,-76.6123462)"
1,1,2015-07-13 10:47:00,Medium,CD,911 NO VOICE,P151941003,600 E FAYETTE ST,"(39.2906737,-76.6071600)"
2,2,2015-07-13 10:42:00,Medium,CD,911 NO VOICE,P151941004,200 E BALTIMORE ST,"(39.2898910,-76.6120720)"
3,3,2015-07-13 10:45:00,Low,CD,PRKG COMPLAINT,P151941005,800 PARK AV,"(39.2985163,-76.6184754)"
4,4,2015-07-13 10:46:00,Medium,SW,AUTO THEFT,P151941006,3500 CLIFTON AV,"(39.3112130,-76.6763150)"
